# Fallstudie Tablet GmbH

## Problemstellung

- Warum war die wirtschaftlich Entwicklung in den letzten beiden Jahren negativ? 
- Welche Faktoren beeinflussten die Situation?
- Wie kann die wirtschaftliche Situation verbessert werden?

## Hypothesen & Fragestellungen (bisher):
- Veränderung der Nachfrage hin zu Tablet Pro & Tablet Mini und weg von Tablet Basic (Silas)
    - Customer Rating als möglicher Indikator (Daniel)
- Downtime als Grund für den geringeren Umsatz in Online-Vertriebskanälen (Vincenzo)
- Sind bestimmte Regionen abhängig von bestimmten Vertriebskanälen/Produkten? (Ali)
- Was verursachte den Umsatzrückgang zum Jahreswechsel? (Ali)

## Einlesen

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df_crm = pd.read_csv("./data_csv/CRM_data.csv")
df_downtime = pd.read_csv("./data_csv/Downtime.csv")
df_preise = pd.read_csv("./data_csv/Preise.csv")

df_a_online = pd.read_csv("./data_csv/RegionA_online.csv")
df_a_phone = pd.read_csv("./data_csv/RegionA_phone.csv")
df_a_store = pd.read_csv("./data_csv/RegionA_store.csv")

df_b_online = pd.read_csv("./data_csv/RegionB_online.csv")
df_b_phone = pd.read_csv("./data_csv/RegionB_phone.csv")
df_b_store = pd.read_csv("./data_csv/RegionB_store.csv")

df_c_online = pd.read_csv("./data_csv/RegionC_online.csv")
df_c_phone = pd.read_csv("./data_csv/RegionC_phone.csv")
df_c_store = pd.read_csv("./data_csv/RegionC_store.csv")

df_d_online = pd.read_csv("./data_csv/RegionD_online.csv")
df_d_phone = pd.read_csv("./data_csv/RegionD_phone.csv")
df_d_store = pd.read_csv("./data_csv/RegionD_store.csv")

## Aufbereiten

### CRM-Daten:

In [5]:
df_crm.head()

,id,vorname,nachname,iban,email,customer_rating
0,0,Curt,Roskoth,DE29557982464763273025,curt.roskoth@gmx.de,7.2
1,1,Till,Klingelhöfer,DE36890407044640624074,till.klingelhöfer@gmail.com,8.4
2,2,Dora,Winkler,DE26144834913572865717,dora.winkler@web.de,6.7
3,3,Tom,Krause,DE16352804742225738584,tom.krause@aol.de,9.4
4,4,Dina,Matthäi,DE24968455090529372560,dina.matthäi@hotmail.de,9.1


In [6]:
df_crm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340000 entries, 0 to 339999
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               340000 non-null  int64  
 1   vorname          340000 non-null  object 
 2   nachname         340000 non-null  object 
 3   iban             340000 non-null  object 
 4   email            340000 non-null  object 
 5   customer_rating  340000 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 15.6+ MB


In [7]:
# Spalten auf Englisch umbenennen
df_crm = df_crm.rename(columns={
    'vorname': 'first_name',
    'nachname': 'last_name',
})

df_crm.head()

,id,first_name,last_name,iban,email,customer_rating
0,0,Curt,Roskoth,DE29557982464763273025,curt.roskoth@gmx.de,7.2
1,1,Till,Klingelhöfer,DE36890407044640624074,till.klingelhöfer@gmail.com,8.4
2,2,Dora,Winkler,DE26144834913572865717,dora.winkler@web.de,6.7
3,3,Tom,Krause,DE16352804742225738584,tom.krause@aol.de,9.4
4,4,Dina,Matthäi,DE24968455090529372560,dina.matthäi@hotmail.de,9.1


### Preisdaten:

In [8]:
df_preise

,Jahr,product,preis
0,"""2021""",Tablet Pro,999
1,"""""20022""",Tablet Pro,999
2,"""2021""",Tablet Mini,680
3,"""""20022""",Tablet Mini,550
4,"""2021""",Tablet Basics,529
5,"""""20022""",Tablet Basics,499


In [9]:
# Umbenennen der Spalte "Jahr" in "year"
df_preise = df_preise.rename(columns={"Jahr": "year",
                                      "preis": "price"})

# Entfernen von Anfuehrungszeichen
df_preise['year'] = df_preise['year'].apply(lambda x: x.strip('"'))  
# Korrigieren der falschen Jahreszahlen
df_preise['year'] = df_preise['year'].replace({'20022': '2022'})  
# Umwandeln der Spalte "jahr" in Datumsformat
df_preise['year'] = pd.to_datetime(df_preise['year'], format='%Y')

# Spalte "jahr" als Jahreszahl extrahieren
df_preise['year'] = df_preise['year'].dt.year

# Werte "Tablet Basics" in "Tablet Basic" ändern
df_preise['product'] = df_preise['product'].replace({'Tablet Basics': 'Tablet Basic'})

df_preise

,year,product,price
0,2021,Tablet Pro,999
1,2022,Tablet Pro,999
2,2021,Tablet Mini,680
3,2022,Tablet Mini,550
4,2021,Tablet Basic,529
5,2022,Tablet Basic,499


### Downtime-Daten:

In [10]:
df_downtime

,Unnamed: 0,Region A,Region B,Region C,Region D
0,2021,0.10,0.01,0.05,0.00
1,2022,0.15,0.15,0.04,0.01


In [11]:
# Erste Spalte in "year" umbenennen
df_downtime = df_downtime.rename(columns={"Unnamed: 0": "year"})

# Spalte "year" in Datumsformat umwandeln
df_downtime['year'] = pd.to_datetime(df_downtime['year'], format='%Y')

# Spalte "year" als Jahreszahl extrahieren
df_downtime['year'] = df_downtime['year'].dt.year

# Umbenennen der Spalten "Region A" bis "Region D" in "region_a" bis "region_d"
df_downtime = df_downtime.rename(columns={"Region A": "region_a", 
                                          "Region B": "region_b", 
                                          "Region C": "region_c", 
                                          "Region D": "region_d"})

df_downtime

,year,region_a,region_b,region_c,region_d
0,2021,0.10,0.01,0.05,0.00
1,2022,0.15,0.15,0.04,0.01


### Transaktionsdaten:

In [12]:
# Hinzufuegen der Spalten "region" und "channel" mit den entsprechenden Werten
df_a_online = df_a_online.assign(region='A', channel='online')
df_a_phone = df_a_phone.assign(region='A', channel='phone')
df_a_store = df_a_store.assign(region='A', channel='store')

df_b_online = df_b_online.assign(region='B', channel='online')
df_b_phone = df_b_phone.assign(region='B', channel='phone')
df_b_store = df_b_store.assign(region='B', channel='store')

df_c_online = df_c_online.assign(region='C', channel='online')
df_c_phone = df_c_phone.assign(region='C', channel='phone')
df_c_store = df_c_store.assign(region='C', channel='store')

df_d_online = df_d_online.assign(region='D', channel='online')
df_d_phone = df_d_phone.assign(region='D', channel='phone')
df_d_store = df_d_store.assign(region='D', channel='store')

In [13]:
# Umbenennen der Spalte "PRODUKT" in "product" in df_b_phone
df_b_phone = df_b_phone.rename(columns={"PRODUKT": "product"})

In [14]:
# Alle Regionen in einen Dataframe zusammenfassen
df_sales = pd.concat([df_a_online, df_a_phone, df_a_store,
                      df_b_online, df_b_phone, df_b_store, 
                      df_c_online, df_c_phone, df_c_store, 
                      df_d_online, df_d_phone, df_d_store])

In [15]:
# Umbenennen der Spalten "Tablet Basic", "Tablet Mini" und "Tablet Pro", "kosten_basic", "kosten_mini" und "kosten_pro"
df_sales = df_sales.rename(columns={
    "Tablet Basic": "tablet_basic",
    "Tablet Mini": "tablet_mini",
    "Tablet Pro": "tablet_pro",
    "kosten_basic": "cost_basic",
    "kosten_mini": "cost_mini",
    "kosten_pro": "cost_pro"
})

In [16]:
# Umwandeln der Spalte "jahr" in Datumsformat
df_sales["date"] = pd.to_datetime(df_sales["date"])

In [17]:
# NaN in den Spalten cost_basic, cost_mini und cost_pro durch 0 ersetzen
df_sales['cost_basic'] = df_sales['cost_basic'].fillna(0)
df_sales['cost_mini'] = df_sales['cost_mini'].fillna(0)
df_sales['cost_pro'] = df_sales['cost_pro'].fillna(0)

In [18]:
# NaN in "discount" durch 1 ersetzen
df_sales['discount'] = df_sales['discount'].fillna(1)

In [19]:
# Zeige Werte der Spalte "product" an
df_sales['product'].unique()

array(['Tablet Basic', 'Tablet Mini', 'Tablet Pro', 'Tablet Basic...'],
      dtype=object)

In [20]:
# Werte "Tablet Basic..." in "Tablet Basic" aendern
df_sales['product'] = df_sales['product'].replace({'Tablet Basic...': 'Tablet Basic'})

In [21]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 340000 entries, 0 to 49438
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            340000 non-null  datetime64[ns]
 1   product         340000 non-null  object        
 2   discount        340000 non-null  float64       
 3   amount          340000 non-null  int64         
 4   tablet_basic    340000 non-null  int64         
 5   tablet_mini     340000 non-null  int64         
 6   tablet_pro      340000 non-null  int64         
 7   cost_basic      340000 non-null  float64       
 8   cost_mini       340000 non-null  float64       
 9   cost_pro        340000 non-null  float64       
 10  transaction_id  340000 non-null  int64         
 11  region          340000 non-null  object        
 12  channel         340000 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(5), object(3)
memory usage: 36.3+ MB


In [22]:
df_sales

,date,product,discount,amount,tablet_basic,tablet_mini,tablet_pro,cost_basic,cost_mini,cost_pro,transaction_id,region,channel
0,2021-11-01,Tablet Basic,1.0,5,1,0,0,170.018118,0.0,0.000000,22,A,online
1,2021-11-01,Tablet Basic,1.0,7,1,0,0,170.019765,0.0,0.000000,24,A,online
2,2021-11-01,Tablet Basic,0.9,7,1,0,0,210.032941,0.0,0.000000,40,A,online
3,2021-11-01,Tablet Basic,1.0,10,1,0,0,170.078236,0.0,0.000000,95,A,online
4,2021-11-01,Tablet Basic,1.0,4,1,0,0,170.085647,0.0,0.000000,104,A,online
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49434,2022-10-31,Tablet Pro,1.0,3,0,0,1,0.000000,0.0,984.987029,339978,D,store
49435,2022-10-31,Tablet Pro,1.0,1,0,0,1,0.000000,0.0,984.988882,339981,D,store
49436,2022-10-31,Tablet Pro,1.0,3,0,0,1,0.000000,0.0,984.991971,339986,D,store
49437,2022-10-31,Tablet Pro,1.0,2,0,0,1,0.000000,0.0,984.995059,339991,D,store


## Transformieren

In [23]:
# df_sales und df_crm zusammenfuegen
df_sales = pd.merge(df_sales, df_crm, left_on='transaction_id', right_on='id', how='inner')

In [24]:
# Spalte "id" entfernen
df_sales = df_sales.drop(columns=['id'])

In [25]:
# Neue Spalten "year", "mmonth" und "week" erstellen
df_sales = df_sales.assign(year=df_sales['date'].dt.year,
                           month=df_sales['date'].dt.month,
                           week=df_sales['date'].dt.isocalendar().week)

In [26]:
# Neue Spalte "price" erstellen, basierend auf den Werten in der Spalte "price" in df_preise
df_sales = df_sales.merge(df_preise, on=["year", "product"], how="left")

In [27]:
# Neue Spalte "cost" erstellen
df_sales['cost'] = df_sales["cost_basic"] + df_sales["cost_mini"] + df_sales["cost_pro"]

In [28]:
# Neue Spalte "cost_total" erstellen
df_sales['cost_total'] = df_sales["cost"] * df_sales["amount"]

In [29]:
# Neue Spalte "revenue" erstellen
df_sales['revenue'] = df_sales['price'] * df_sales['amount'] * df_sales['discount']

In [30]:
# Profit pro Stueck berechnen
df_sales["profit_per_piece"] = (df_sales["price"] * df_sales["discount"]) - df_sales["cost"]

In [31]:
# Profit des gesamten Verkaufs berechnen
df_sales["profit_total"] = df_sales["profit_per_piece"] * df_sales["amount"]

In [32]:
# Alle Spalten anzeigen lassen
pd.set_option('display.max_columns', None)

df_sales

,date,product,discount,amount,tablet_basic,tablet_mini,tablet_pro,cost_basic,cost_mini,cost_pro,transaction_id,region,channel,first_name,last_name,iban,email,customer_rating,year,month,week,price,cost,cost_total,revenue,profit_per_piece,profit_total
0,2021-11-01,Tablet Basic,1.0,5,1,0,0,170.018118,0.0,0.000000,22,A,online,Friedhilde,Kuhl,DE23093007879691629842,friedhilde.kuhl@yahoo.de,7.1,2021,11,44,529,170.018118,850.090589,2645.0,358.981882,1794.909411
1,2021-11-01,Tablet Basic,1.0,7,1,0,0,170.019765,0.0,0.000000,24,A,online,Heide-Marie,Fischer,DE26303459399974289421,heide-marie.fischer@gmx.de,5.9,2021,11,44,529,170.019765,1190.138353,3703.0,358.980235,2512.861647
2,2021-11-01,Tablet Basic,0.9,7,1,0,0,210.032941,0.0,0.000000,40,A,online,Ester,Pruschke,DE04679916372817044326,ester.pruschke@gmail.com,7.5,2021,11,44,529,210.032941,1470.230589,3332.7,266.067059,1862.469411
3,2021-11-01,Tablet Basic,1.0,10,1,0,0,170.078236,0.0,0.000000,95,A,online,Klaus,Blümel,DE54334149840465696270,klaus.blümel@aol.de,6.6,2021,11,44,529,170.078236,1700.782355,5290.0,358.921764,3589.217645
4,2021-11-01,Tablet Basic,1.0,4,1,0,0,170.085647,0.0,0.000000,104,A,online,Waldemar,Fritsch,DE54862551075047386387,waldemar.fritsch@web.de,7.3,2021,11,44,529,170.085647,680.342589,2116.0,358.914353,1435.657411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339995,2022-10-31,Tablet Pro,1.0,3,0,0,1,0.000000,0.0,984.987029,339978,D,store,Carla,Klemm,DE95736837564420121887,carla.klemm@web.de,7.6,2022,10,44,999,984.987029,2954.961088,2997.0,14.012971,42.038912
339996,2022-10-31,Tablet Pro,1.0,1,0,0,1,0.000000,0.0,984.988882,339981,D,store,Hans-Uwe,Döring,DE96536545575195298222,hans-uwe.döring@yahoo.de,7.6,2022,10,44,999,984.988882,984.988882,999.0,14.011118,14.011118
339997,2022-10-31,Tablet Pro,1.0,3,0,0,1,0.000000,0.0,984.991971,339986,D,store,Irene,Aumann,DE83463083316300535191,irene.aumann@hotmail.de,8.3,2022,10,44,999,984.991971,2954.975912,2997.0,14.008029,42.024088
339998,2022-10-31,Tablet Pro,1.0,2,0,0,1,0.000000,0.0,984.995059,339991,D,store,Jolanthe,Spieß,DE73981381785498034616,jolanthe.spieß@gmx.de,9.2,2022,10,44,999,984.995059,1969.990118,1998.0,14.004941,28.009882


In [33]:
# Sortieren nach Datum
df_sales.sort_values(by=['transaction_id'], inplace=True)

# Reset der Index-Spalte
df_sales.reset_index(drop=True, inplace=True)

In [36]:
df_sales

,date,product,discount,amount,tablet_basic,tablet_mini,tablet_pro,cost_basic,cost_mini,cost_pro,transaction_id,region,channel,first_name,last_name,iban,email,customer_rating,year,month,week,price,cost,cost_total,revenue,profit_per_piece,profit_total
0,2021-11-01,Tablet Pro,1.0,4,0,0,1,0.000000,0.0,695.000000,0,C,store,Curt,Roskoth,DE29557982464763273025,curt.roskoth@gmx.de,7.2,2021,11,44,999,695.000000,2780.000000,3996.0,304.000000,1216.000000
1,2021-11-01,Tablet Basic,1.0,3,1,0,0,170.000824,0.0,0.000000,1,D,online,Till,Klingelhöfer,DE36890407044640624074,till.klingelhöfer@gmail.com,8.4,2021,11,44,529,170.000824,510.002471,1587.0,358.999176,1076.997529
2,2021-11-01,Tablet Basic,0.9,5,1,0,0,200.001647,0.0,0.000000,2,C,phone,Dora,Winkler,DE26144834913572865717,dora.winkler@web.de,6.7,2021,11,44,529,200.001647,1000.008235,2380.5,276.098353,1380.491765
3,2021-11-01,Tablet Basic,1.0,2,1,0,0,170.002471,0.0,0.000000,3,B,online,Tom,Krause,DE16352804742225738584,tom.krause@aol.de,9.4,2021,11,44,529,170.002471,340.004941,1058.0,358.997529,717.995059
4,2021-11-01,Tablet Basic,1.0,2,1,0,0,150.003294,0.0,0.000000,4,C,online,Dina,Matthäi,DE24968455090529372560,dina.matthäi@hotmail.de,9.1,2021,11,44,529,150.003294,300.006588,1058.0,378.996706,757.993412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339995,2022-10-31,Tablet Pro,1.0,2,0,0,1,0.000000,0.0,864.997529,339995,D,phone,Martin,Heinrich,DE06833178600136945770,martin.heinrich@aol.de,8.4,2022,10,44,999,864.997529,1729.995059,1998.0,134.002471,268.004941
339996,2022-10-31,Tablet Pro,1.0,3,0,0,1,0.000000,0.0,864.998147,339996,D,phone,Hartwig,Koch II,DE66942502046709852454,hartwig.koch_ii@yahoo.de,8.2,2022,10,44,999,864.998147,2594.994441,2997.0,134.001853,402.005559
339997,2022-10-31,Tablet Pro,1.0,2,0,0,1,0.000000,0.0,944.998765,339997,D,store,Reza,Boucsein,DE35718077975432977896,reza.boucsein@web.de,9.9,2022,10,44,999,944.998765,1889.997529,1998.0,54.001235,108.002471
339998,2022-10-31,Tablet Pro,1.0,4,0,0,1,0.000000,0.0,864.999382,339998,B,store,Thekla,Vollbrecht,DE65129187433166256313,thekla.vollbrecht@web.de,8.7,2022,10,44,999,864.999382,3459.997529,3996.0,134.000618,536.002471


## Visualisieren

In [46]:
df_sales.groupby(["product","year", "month"]).agg({"customer_rating" : {"mean", "median", "min", "max"}, "amount" : "count"}).round(2)
#linear regression between customer_rating and amount


customer_rating                  amount
                                 median  mean   max  min  count
product      year month                                        
Tablet Basic 2021 11                6.6  6.79  10.0  3.7  11406
                  12                6.2  6.39  10.0  3.4  11077
             2022 1                 5.9  6.04  10.0  3.0   9897
                  2                 5.6  5.70  10.0  2.7   8419
                  3                 5.3  5.42  10.0  2.8   8149
                  4                 5.1  5.20  10.0  2.2   7071
                  5                 4.8  4.92  10.0  2.1   6497
                  6                 4.6  4.73  10.0  2.0   5158
                  7                 4.5  4.56  10.0  1.8   4509
                  8                 4.3  4.42  10.0  2.0   3556
                  9                 4.2  4.31   9.8  1.6   2538
                  10                4.1  4.23   8.4  1.6   1723
Tablet Mini  2021 11                8.8  8.75  10.0  5.2   8986
                  12                8.6  8.61  10.0  5.3   9169
             2022 1                 8.5  8.49  10.0  5.2   9335
                  2                 8.3  8.37  10.0  4.8   8645
                  3                 8.2  8.23  10.0  4.3   9351
                  4                 8.0  8.10  10.0  4.3   9099
                  5                 7.9  7.94  10.0  4.3   9420
                  6                 7.7  7.83  10.0  4.4   9003
                  7                 7.6  7.68  10.0  3.9   9227
                  8                 7.4  7.52  10.0  3.3   9349
                  9                 7.3  7.40  10.0  4.0   9046
                  10                7.2  7.29  10.0  4.2   9370
Tablet Pro   2021 11                7.8  7.93  10.0  5.0   3061
                  12                7.8  7.94  10.0  4.5   4928
             2022 1                 7.8  7.96  10.0  5.0   6876
                  2                 7.9  8.02  10.0  4.7   7601
                  3                 8.0  8.05  10.0  4.7  10033
                  4                 8.0  8.11  10.0  5.0  11309
                  5                 8.1  8.17  10.0  4.5  13356
                  6                 8.1  8.21  10.0  4.5  14962
                  7                 8.2  8.31  10.0  4.7  17062
                  8                 8.3  8.38  10.0  5.0  18595
                  9                 8.4  8.48  10.0  5.3  20088
                  10                8.5  8.57  10.0  5.1  22129

## Modellieren

## Kommunizieren